# [프렉티컴] : 아동 - AI 대화 요약 모델 만들기
* 데이터 출처: AIHUB-아동.청소년상담데이터 활용 (텍스트 데이터만 활용)

In [18]:
'''# 하이퍼 클로바 멀티 모델 다운 받기 이건 그냥 참고용
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor

model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B" #"naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"  # HyperCLOVAX-SEED-Text-Instruct-0.5B  HyperCLOVAX-SEED-Vision-Instruct-3B
local_dir =  "./HyperCLOVAX-SEED-Text-Instruct-1.5B" # "./HyperCLOVAX-SEED-Text-Instruct-0.5B" # HyperCLOVAX-SEED-Text-Instruct-0.5B  HyperCLOVAX-SEED-Vision-Instruct-3B

AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, cache_dir=local_dir)
AutoTokenizer.from_pretrained(model_name, cache_dir=local_dir)
AutoProcessor.from_pretrained(model_name, trust_remote_code=True, cache_dir=local_dir)'''

'# 하이퍼 클로바 멀티 모델 다운 받기 이건 그냥 참고용\nfrom transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor\n\nmodel_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B" #"naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"  # HyperCLOVAX-SEED-Text-Instruct-0.5B  HyperCLOVAX-SEED-Vision-Instruct-3B\nlocal_dir =  "./HyperCLOVAX-SEED-Text-Instruct-1.5B" # "./HyperCLOVAX-SEED-Text-Instruct-0.5B" # HyperCLOVAX-SEED-Text-Instruct-0.5B  HyperCLOVAX-SEED-Vision-Instruct-3B\n\nAutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, cache_dir=local_dir)\nAutoTokenizer.from_pretrained(model_name, cache_dir=local_dir)\nAutoProcessor.from_pretrained(model_name, trust_remote_code=True, cache_dir=local_dir)'

# STEP 0. 구글마운트 / 경로 설정 / 패키지 로드

In [1]:
# 구글 마운트
from google.colab import drive
drive.mount('/content/drive')

# 패키지로드
import glob
import json
import pandas as pd
from pathlib import Path
import random
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import torch


# 경로설정
ROOT_DIR = Path('/content/drive/MyDrive/개인폴더/서강대/5. 2025-1학기/3. 프랙티컴/1.모델학습/[AIHUB]심리상담데이터') # 루트 경로
TYPE_DIR = '1. (상태파악) 아동상담데이터' # 데이터 있는 폴더
TRAIN_DIR = ROOT_DIR / TYPE_DIR / '기초자료' # 훈련데이터
VAL_DIR = ROOT_DIR / TYPE_DIR / '검증자료' # 검증데이터

Mounted at /content/drive


# STEP 1. 데이터 로드

- 훈련데이터: 2876개 데이터 (실제 아동과 임상심리사 대화 및 임상심리사 소견)

In [4]:
# 파일 위치 설정
TRAIN_FILES = list(TRAIN_DIR.glob('*.json')) # 훈련데이터 경로들
VAL_FILES = list(VAL_DIR.glob('*.json')) # 검증데이터 경로들

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# 파일 불러와서 프롬프트 형식 데이터 정리 및 의견 리턴
def load_json(file_path):
  # 파일 불러오기
  with open(file_path, 'r', encoding='utf-8') as f:
      data = json.load(f)
      # 각 영역별 내용 정리
      short_opinion= data['info']['임상가 종합소견'] # 종합소견
      risk_opinion = data['info']['위기단계'] # 위기단계
      abuse_opinion = data['info']['학대의심'] # 학대의심

      # 데이터 중에서 내담자(user)와 상담자(assistant) 프롬프트 형식으로 출력
      # 해당 데이터는 추후 아동 대화 llm에 lora에 자연스러운 발화에 쓰임
      full_list = []
      for master_data in data['list']:
        sub_list = [{"role": "system", "content": "너는 친절한 상담자야."}]
        for idx, sub_data in enumerate(master_data['list']):
          try:
            for text_list in sub_data['audio']:

              if text_list['type'] == 'A':
                sub_list.append({'role':'user','content':text_list['text']})
              else:
                sub_list.append({'role':'assistant','content':text_list['text']})
          except:
            pass
        # 프롬프트 틀에 맞추기 위해서 user 제거
        sub_list = sub_list[:-1].copy()
        sub_list[-1] =  {"role": "assistant", "content": "오늘 이야기 해주어서 고마워"}
        full_list.append(sub_list)

  return full_list, short_opinion, risk_opinion, abuse_opinion #프롬프트 형식, 임상가소견, 위기단계, 학대의심

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

train_df = list() # 정리된 데이터 저장

# 훈련데이터 추출
for idx, file in enumerate(TRAIN_FILES):
  if idx % 500 == 0:
    print(f'{idx+1}번째 파일')
  raw_dict = {}
  full_list, short_opinion, risk_opinion, abuse_opinion = load_json(file)
  raw_dict['prompt'] = full_list # 프롬프트
  raw_dict['short_opinion'] = short_opinion # 임상가 소견
  raw_dict['risk_opinion'] = risk_opinion # 위기 단계
  raw_dict['abuse_opinion'] = abuse_opinion # 학대 의심
  train_df.append(raw_dict)

train_df = pd.DataFrame(train_df)

val_df = list() # 정리된 데이터 저장

# 검증용 데이터 추출
for file in VAL_FILES:
  raw_dict = {}
  full_list, short_opinion, risk_opinion, abuse_opinion = load_json(file)
  raw_dict['prompt'] = full_list # 프롬프트
  raw_dict['short_opinion'] = short_opinion # 임상가 소견
  raw_dict['risk_opinion'] = risk_opinion # 위기 단계
  raw_dict['abuse_opinion'] = abuse_opinion # 학대 의심
  val_df.append(raw_dict)

val_df = pd.DataFrame(val_df)
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# 값 출력
print('훈련데이터 데이터 구조 예시')
display(train_df.head(3))

print('\n위기단계 개수' )
display(pd.DataFrame(train_df['risk_opinion'].value_counts()).T)

print('\n학대의심 개수')
display(pd.DataFrame(train_df['abuse_opinion'].value_counts()).T)

print('\n\n--------------------------------------')
print('검증데이터 데이터 구조 예시')
display(val_df.head(3))

print('\n위기단계 개수' )
display(pd.DataFrame(val_df['risk_opinion'].value_counts()).T)

print('\n학대의심 개수')
display(pd.DataFrame(val_df['abuse_opinion'].value_counts()).T)

1번째 파일
501번째 파일
1001번째 파일
1501번째 파일
2001번째 파일
2501번째 파일
훈련데이터 데이터 구조 예시


,prompt,short_opinion,risk_opinion,abuse_opinion
0,"[[{'role': 'system', 'content': '너는 친절한 상담자야.'...","아동은 신체적으로 건강한 상태이며, 심각한 신체적 손상이나 통증은 보고되지 않았습니...",관찰필요,(해당 없음)
1,"[[{'role': 'system', 'content': '너는 친절한 상담자야.'...",위 아동은 심리적 원인으로 인한 통증을 호소하고 있고 아버지로부터 혐오나 무시의 감...,상담필요,성학대
2,"[[{'role': 'system', 'content': '너는 친절한 상담자야.'...",아동은 통증이나 신체적 손상 없이 건강한 상태를 유지하고 있습니다. 분노와 짜증을 ...,상담필요,성학대



위기단계 개수


risk_opinion,정상군,학대의심,응급,관찰필요,상담필요
count,598,587,576,564,551



학대의심 개수


...

,prompt,short_opinion,risk_opinion,abuse_opinion
0,"[[{'role': 'system', 'content': '너는 친절한 상담자야.'...",종합적으로 해당 아동은 가정 내 아버지와의 관계에서 학대와 폭력 가능성이 의심되고 ...,응급,신체학대
1,"[[{'role': 'system', 'content': '너는 친절한 상담자야.'...","해당 아동의 상황을 종합적으로 고려한 결과, 아동은 신체적 손상 없이 건강한 상태로...",상담필요,(해당 없음)
2,"[[{'role': 'system', 'content': '너는 친절한 상담자야.'...","아동은 일부 통증을 경험하고 있으며, 이는 늦게 취침하는 생활 습관과 관련이 있을 ...",관찰필요,(해당 없음)



위기단계 개수


risk_opinion,응급,상담필요,관찰필요,정상군,학대의심
count,96,70,70,67,57



학대의심 개수


abuse_opinion,(해당 없음),신체학대,정서학대,성학대,방임
count,203,56,40,36,25


#STEP 2. LLM 불러오기 (Naver HyperCLOVA SEED 0.5B)모델

🔹 개요
- HyperCLOVA SEED 0.5B는 NAVER에서 공개한 한국어 특화 경량 언어 모델로, 총 5억 개 파라미터(0.5B)를 가진 **Causal Language Model (CLM)**입니다.
- 학습 데이터와 사전 구조 모두 한국어 중심으로 설계되어 한국어 환경에서의 추론 성능과 효율성에 초점을 둡니다.

🔹 주요 특징
- 모델 크기: 0.5B (약 5억 파라미터)
- 모델 유형: Causal Language Model (AutoModelForCausalLM)
- 지원 토크나이저: AutoTokenizer 기반, SentencePiece 사용
- 학습 목적: Instruction Tuning, Chat, Summarization 등 한국어 응용

🔹 활용 목적
- 경량 모델로서 일반적인 GPU 환경에서 추론 및 미세조정(fine-tuning)이 가능함
- LoRA 기반 미세조정을 통해 대규모 모델 대비 훨씬 가볍고 빠른 커스터마이징이 가능
- ChatGPT 스타일 대화 응답, 한국어 질의응답, 요약, 분류 등에 활용됨

In [2]:
# 파일 경로
MODEL_DIR = ROOT_DIR / 'HyperCLOVAX-SEED-Text-Instruct-0.5B/models--naver-hyperclovax--HyperCLOVAX-SEED-Text-Instruct-0.5B/snapshots/b8c527c20d8fede1190c0b9c5e9821f30f42498a'

# 토큰나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

# 모델 로딩 (학습/추론 시 필요)
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, trust_remote_code=True).to(device='cuda')

#STEP 3. AI-화자 대화턴 -> 요약문 만들기 (lora 적용)

🔹 목적
- 사용자(user)와 AI(assistant) 간 여러 차례의 대화를 기반으로, 마지막에 “그 동안의 이야기만 요약해줘”라는 요청에 대해 적절한 요약을 생성하는 모델을 학습시킨다.

🔹 데이터 구성
- 데이터는 리스트 구조로, 각 요소는 여러 턴으로 구성된 대화와 이에 대한 요약문으로 이루어져 있다.
- 대화는 user와 assistant의 발화를 교차로 포함하며, system 역할은 무시된다.
- 각 대화는 <|user|>와 <|assistant|> 토큰으로 구분되고, 마지막에 요약 요청 문장이 추가된다. 이 전체가 모델 입력이 되며, 정답 레이블은 해당 요약문이다.

🔹 학습 방식
- 입력 텍스트는 "대화 + 요약 요청 문장"
- 출력(라벨)은 실제 요약문
- 라벨에서는 요약 요청 전까지의 입력 부분은 손실 계산에서 제외 (-100 처리)
- LoRA (Low-Rank Adaptation) 방식으로 기존 언어모델에 경량 어댑터를 붙여 학습 진행
- 전체 모델을 학습하지 않고 일부 층만 미세조정하기 때문에 빠르고 효율적임

🔹 학습 목적
- 한국어로 된 다중 대화 흐름을 이해하고, 요약하는 능력 학습, 특히    **심리학 관점**  임상데이터 요약 가능
- 향후 추론 시에도 동일한 형식의 입력(대화 + 요약 요청)을 통해 자연스러운 요약 생성 가능
- 예: "<|user|>\n안녕?\n<|assistant|>\n안녕하세요!\n<|user|>\n오늘 있었던 일을 정리해줘" → 모델이 요약을 생성

In [15]:
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# [준비단계] 학습에 필요한 형태로 만들기
# 3중 리스트 구조로 이루어져 있어 이중 루프문으로 요소추출
def convert_conversation(convo_group, summary):
    lines = []
    for episode in convo_group:
        for turn in episode:
            if turn["role"] == "user":
                lines.append("<|user|>\n" + turn["content"])
            elif turn["role"] == "assistant":
                lines.append("<|assistant|>\n" + turn["content"])
            # system은 무시

    full_input = "\n".join(lines) + "\n<|user|>\n그 동안의 이야기만 요약해줘."

    return {
        "input": full_input,
        "short_opinion": summary
    }
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# 함수 적용해서 train, val 분리
train_list = [ convert_conversation(prompt,short_opinion) for prompt, short_opinion in zip(train_df['prompt'],train_df['short_opinion'])]
val_list = [ convert_conversation(prompt,short_opinion) for prompt, short_opinion in zip(val_df['prompt'],val_df['short_opinion'])]

In [17]:
# 데이터 예시
val_list[0]

{'input': '<|assistant|>\n최근에 아픈 곳이 있어?\n<|user|>\n네. 머리가 아파요.\n<|assistant|>\n머리가 어떻게 아파?\n<|user|>\n그냥 지끈거려요. 뭔가 긴장되는 상황이 생기면 머리가 아픈 거 같아요.\n<|assistant|>\n이전에도 해당 부위가 자주 아팠어?\n<|user|>\n네. 자주 그래요. 근데 오래 지속되지는 않아요.\n<|assistant|>\n몸 여기저기에 반창고가 많네. 언제 다친 거야?\n<|user|>\n며칠 전에 아빠한테 혼나서 다쳤어요.\n<|assistant|>\n오늘 이야기 해주어서 고마워\n<|assistant|>\n친구는 뭐 할 때가 즐거워?\n<|user|>\n잘 모르겠어요.\n<|assistant|>\n즐겁다고 느끼지 못한 게 언제부터야?\n<|user|>\n그것도 잘 모르겠어요.\n<|assistant|>\n최근 일주일 동안 짜증이나 화가 난 적 있어?\n<|user|>\n네. 화난 적 있어요.\n<|assistant|>\n무엇 때문에 화가 났어?\n<|user|>\n아빠가 무서워서 아무것도 못하는 제 자신에게 화가 났어요.\n<|assistant|>\n그런 일이 자주 일어나?\n<|user|>\n네.\n<|assistant|>\n오늘 이야기 해주어서 고마워\n<|assistant|>\n몇 시에 자고 몇 시에 일어나?\n<|user|>\n새벽 한 시쯤 자고 일곱 시에 일어나요.\n<|assistant|>\n아침에 일어났을 때 몸 상태는 어때?\n<|user|>\n안 좋아요. 전날에 다치거나 했을 때는 몸이 아프고, 아니면 그냥 안 좋아요. 힘도 없고요.\n<|assistant|>\n오늘 이야기 해주어서 고마워\n<|assistant|>\n평소 아빠는 어떤 표정을 자주 지으셔?\n<|user|>\n화내는 표정이요.\n<|assistant|>\n아빠가 왜 그러시는 거 같아?\n<|user|>\n그건 모르겠지만 항상 화나있어요.\n<|assistant|>\n아빠가 그런 표정

In [18]:
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# [모델구성] LORA 설정 적용 후 기존 Naver 모델과 결합

# LORA 설정 적용
lora_config = LoraConfig(
    r=8,  # LoRA의 rank 값. 낮을수록 경량화되고, 높을수록 표현력 증가. 일반적으로 4~16 사이 사용.

    lora_alpha=16,  # 학습 시 scaling factor. 보통 r과 같은 값이거나 r보다 큰 값을 사용함.
                    # 출력값은 LoRA output * (alpha / r)로 스케일됨.

    task_type=TaskType.CAUSAL_LM,  # 해당 모델이 수행하는 태스크 종류.
                                   # Causal Language Model (AutoModelForCausalLM)에 적용할 것이므로 이 값 사용.

    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # LoRA를 적용할 대상 모듈.
                                                              # 주로 attention 내부의 query/key/value/output projection에 적용함.

    lora_dropout=0.1,  # 학습 중 LoRA layer에 적용할 dropout 확률. 과적합 방지 목적.

    bias="none"  # bias 학습 여부.
                 # "none"은 기존 모델의 bias는 그대로 유지하고, LoRA에서는 bias를 따로 학습하지 않음.
)

# LORA 설정과 기존 모델과 결합
lora_model = get_peft_model(model, lora_config)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# [토큰나이저 구성] 데이터를 학습시키도록 변환을 시킨 뒤에 토큰나이저화

dataset = Dataset.from_list(train_list) # HuggingFace Dataset 객체로 변환

def preprocess(example):
    # 입력 문장(prompt)과 정답 요약(summary)을 가져옴
    prompt = example["input"]
    summary = example["short_opinion"]

    # 전체 입력 = prompt + "\n" + summary 형태로 구성
    full_text = prompt + "\n" + summary

    # 모델 입력용: prompt만 토크나이징 (학습 시 정답이 아님)
    inputs = tokenizer(prompt, truncation=True, padding="max_length", max_length=2048) # 해당 숫자를 충분히 해야 모든 입력데이터를 학습 가능

    # 모델 학습용: 정답 summary까지 포함한 전체 문장을 토크나이징
    full = tokenizer(full_text, truncation=True, padding="max_length", max_length=2048) # 해당 숫자를 충분히 해야 모든 입력데이터를 학습 가능

    # prompt 길이를 기준으로, 그 이전 토큰은 -100으로 설정 → 학습에서 무시됨
    input_len = len(inputs["input_ids"])
    labels = full["input_ids"]
    labels = [-100 if i < input_len else tok_id for i, tok_id in enumerate(labels)]

    # 모델 학습을 위한 포맷 반환: 전체 input, attention_mask, 정답 labels
    return {
        "input_ids": full["input_ids"],
        "attention_mask": full["attention_mask"],
        "labels": labels
    }

# dataset.map(preprocess): 원본 dataset에 대해 preprocess 함수를 적용하여 모델 학습에 맞는 형식(input_ids, attention_mask, labels)으로 변환
tokenized_dataset = dataset.map(preprocess)

# padding 시 사용할 토큰을 eos_token(문장 끝 토큰)으로 설정
# → tokenizer가 pad_token을 기본적으로 갖고 있지 않은 경우 오류 발생 가능, 따라서 명시적으로 설정
tokenizer.pad_token = tokenizer.eos_token

# tokenizer가 처리할 수 있는 최대 토큰 길이를 모델의 position embedding 한계값으로 맞춤
# → 예: 대부분의 모델은 2048 또는 4096 토큰까지만 입력 가능하므로 이를 자동으로 맞춤
tokenizer.model_max_length = lora_model.base_model.config.max_position_embeddings

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# [학습 및 저장] LORA 학습 후 LORA 학습 후 가중치 저장

# 학습 설정
training_args = TrainingArguments(
    output_dir=ROOT_DIR / TYPE_DIR / "lora_results",  # 학습 결과 및 체크포인트 저장 경로
    per_device_train_batch_size=1,                    # 디바이스당 배치 크기 (메모리 제한 고려)
    num_train_epochs=1,                               # 전체 데이터셋을 몇 번 반복할지 (에폭 수)
    logging_steps=100,                                # 100 step마다 로그 출력
    save_strategy="epoch",                            # 에폭 단위로 체크포인트 저장
    save_total_limit=2,                               # 최대 2개의 체크포인트만 저장 (디스크 공간 절약)
    warmup_steps=10,                                  # 워밍업 스텝 수 (learning rate가 처음에 서서히 증가)
    learning_rate=5e-5,                               # 학습률 설정
    weight_decay=0.01,                                # 가중치 감소 (과적합 방지)
    fp16=torch.cuda.is_available(),                   # GPU 사용 가능하면 mixed precision(FP16) 활성화
    report_to="none"                                  # 로그를 외부 툴(예: WandB)에 보고하지 않음
)

# Trainer 정의
trainer = Trainer(
    model=lora_model,                                                  # 학습할 모델 (LoRA가 적용된 모델)
    args=training_args,                                                # 위에서 정의한 학습 설정
    train_dataset=tokenized_dataset,                                   # 학습 데이터셋
    tokenizer=tokenizer,                                               # tokenizer 정보 포함 (학습 중 padding 등 처리에 필요)
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),  # 데이터 정렬 방식 설정 (MLM 아님 → causal LM용)
)

# 실제 학습 수행
trainer.train()

# LoRA adapter만 저장  (base model은 저장하지 않음)
lora_model.save_pretrained(ROOT_DIR / TYPE_DIR / "lora_only_adapter")
tokenizer.save_pretrained(ROOT_DIR / TYPE_DIR / "lora_only_adapter")

Map:   0%|          | 0/2876 [00:00<?, ? examples/s]

<ipython-input-18-eb5526223ede>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,1.722000
200,1.067700
300,0.882300
400,0.881900
500,0.819500
600,0.795700
700,0.805200
800,0.741300
900,0.784200
1000,0.749800


('/content/drive/MyDrive/개인폴더/서강대/5. 2025-1학기/3. 프랙티컴/1.모델학습/[AIHUB]심리상담데이터/1. (상태파악) 아동상담데이터/lora_only_adapter/tokenizer_config.json',
 '/content/drive/MyDrive/개인폴더/서강대/5. 2025-1학기/3. 프랙티컴/1.모델학습/[AIHUB]심리상담데이터/1. (상태파악) 아동상담데이터/lora_only_adapter/special_tokens_map.json',
 '/content/drive/MyDrive/개인폴더/서강대/5. 2025-1학기/3. 프랙티컴/1.모델학습/[AIHUB]심리상담데이터/1. (상태파악) 아동상담데이터/lora_only_adapter/chat_template.jinja',
 '/content/drive/MyDrive/개인폴더/서강대/5. 2025-1학기/3. 프랙티컴/1.모델학습/[AIHUB]심리상담데이터/1. (상태파악) 아동상담데이터/lora_only_adapter/vocab.json',
 '/content/drive/MyDrive/개인폴더/서강대/5. 2025-1학기/3. 프랙티컴/1.모델학습/[AIHUB]심리상담데이터/1. (상태파악) 아동상담데이터/lora_only_adapter/merges.txt',
 '/content/drive/MyDrive/개인폴더/서강대/5. 2025-1학기/3. 프랙티컴/1.모델하

#STEP 4. LORA 가중치를 불러온 후 테스트 하기

In [21]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch

# 기존 모델  경로
MODEL_DIR = ROOT_DIR / 'HyperCLOVAX-SEED-Text-Instruct-0.5B/models--naver-hyperclovax--HyperCLOVAX-SEED-Text-Instruct-0.5B/snapshots/b8c527c20d8fede1190c0b9c5e9821f30f42498a'

# (1) LoRA adapter 디렉토리
adapter_path = ROOT_DIR / TYPE_DIR / "lora_only_adapter"

# (2) LoRA에 포함된 base 모델 정보 확인
peft_config = PeftConfig.from_pretrained(adapter_path)

# (3) base 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, trust_remote_code=True).to(device='cuda')

# (4) LoRA adapter 적용
model = PeftModel.from_pretrained(base_model, adapter_path).to("cuda")
model.eval()

# (5) tokenizer는 adapter 경로에서 불러도 무방
tokenizer = AutoTokenizer.from_pretrained(adapter_path)
tokenizer.pad_token = tokenizer.eos_token

In [22]:
# 5. val_list에서 샘플 하나 선택
sample = val_list[3]
input_text = sample["input"]

# 6. 토크나이즈 및 추론
inputs = tokenizer(input_text, return_tensors="pt", padding=True)
inputs = {k: v.to("cuda" if torch.cuda.is_available() else "cpu") for k, v in inputs.items()}

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=1024,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

result = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
print("요약:", result)

요약:  해당 아동은 신체적 손상이 관찰되지 않으며, 정서적 학대나 학대 징후는 관찰되지 않습니다. 또한, 아동은 건강한 상태로 판단되며, 아동의 상태는 정상으로 판단됩니다. 그러나 아동의 어머니가 아동에게 무서운 표정을 짓는 것으로 보아, 아동의 정서적 학대 가능성이 있을 수 있습니다. 아동의 어머니는 아동의 안전을 위해 적절한 돌봄과 지원을 제공해야 합니다. 또한, 아동의 형제자매와 친구들과의 관계에서 갈등이 발생할 수 있으므로, 아동의 정서적 안정을 위해 적절한 대처가 필요합니다. 아동의 상태는 정상으로 판단되며, 아동의 상태는 정상으로 판단됩니다. 그러나 아동의 어머니가 아동의 안전을 위해 적절한 돌봄과 지원을 제공해야 합니다. 또한, 아동의 형제자매와 친구들과의 관계에서 갈등이 발생할 수 있으므로, 아동의 정서적 안정을 위해 적절한 대처가 필요합니다. 아동의 상태는 정상으로 판단되며, 아동의 상태는 정상으로 판단됩니다. 그러나 아동의 어머니가 아동의 안전을 위해 적절한 돌봄과 지원을 제공해야 합니다. 또한, 아동의 형제자매와 친구들과의 관계에서 갈등이 발생할 수 있으므로, 아동의 정서적 안정을 위해 적절한 대처가 필요합니다. 아동의 상태는 정상으로 판단되며, 아동의 상태는 정상으로 판단됩니다. 그러나 아동의 어머니가 아동의 안전을 위해 적절한 돌봄과 지원을 제공해야 합니다. 또한, 아동의 형제자매와 친구들과의 관계에서 갈등이 발생할 수 있으므로, 아동의 정서적 안정을 위해 적절한 대처가 필요합니다. 아동의 상태는 정상으로 판단되며, 아동의 상태는 정상으로 판단됩니다. 그러나 아동의 어머니가 아동의 안전을 위해 적절한 돌봄과 지원을 제공해야 합니다. 또한, 아동의 형제자매와 친구들과의 관계에서 갈등이 발생할 수 있으므로, 아동의 정서적 안정을 위해 적절한 대처가 필요합니다. 아동의 상태는 정상으로 판단되며, 아동의 상태는 정상으로 판단됩니다. 그러나 아동의 어머니가 아동의 안전을 위해 적절한 돌봄과 지원을 제공해야 합니다. 또한, 아동의 형제자매와 친구